In [9]:
# # load API key
import os
from dotenv import load_dotenv

# to log
import logging

# parse json file with transcribed videos
import json

# import openai

from llama_index import Document
from llama_index.llms import OpenAI
from llama_index.evaluation import DatasetGenerator, RelevancyEvaluator, FaithfulnessEvaluator

from llama_index import StorageContext, load_index_from_storage
from llama_index import ServiceContext

import pandas as pd
from llama_index.llms import OpenAI

In [ ]:
    # get the API key to the variable
    load_dotenv()

    try:
        openai.api_key = os.getenv("API_KEY")
    except:
        pass

In [3]:
# from llama_index.node_parser import SimpleNodeParser
# from data_pipelines.parser_transcribe import ParserTranscribe, get_video_urls

In [10]:
path = '/Users/dmitrii.shiriaev/Library/CloudStorage/OneDrive-KarolinskaInstitutet/Code/simulator-ml/KarpovAI'

# set the logging
logging.basicConfig(filename=f'{path}/question_generation/question_generation.log',  # Name of the log file
                    level=logging.DEBUG,     # Logging level
                    format='%(asctime)s - %(levelname)s - %(message)s',  # Format of log messages
                    filemode='w')  # Overwrite the log file on each script run

logging.debug('Run started')

In [11]:
with open(f"{path}/data/video_info.json", "r", encoding="utf-8") as f:
    data = json.load(f)

data = data[:1]

In [12]:
docs = [
    Document(
        text=d["text"][0],
        metadata={"url": d["url"][0], "title": d["title"][0]},
    )
    for d in data
]


In [14]:
text = data[0]['text']

logging.debug(f'example of text object: {text}')


In [16]:
data_generator = DatasetGenerator.from_documents(
    documents=docs,
    num_questions_per_chunk=2,
    question_gen_query="На русском языке"
)


ValueError: No API key found for OpenAI.
Please set either the OPENAI_API_KEY environment variable or openai.api_key prior to initialization.
API keys can be found or created at https://platform.openai.com/account/api-keys


In [ ]:
eval_questions = data_generator.generate_questions_from_nodes()